In [3]:
import pandas as pd
import numpy as np
import os
import cv2

In [25]:
os.makedirs('Output/', exist_ok=True)
os.makedirs('data', exist_ok=True)

label_dir = "Output/label/"
image_dir = "Output/images2/"

for dd in [label_dir, image_dir]:
    for ss in ["train", "valid", "test"]:
        print(os.path.join(dd, ss))
        os.makedirs(os.path.join(dd, ss), exist_ok=True)
        
annotation = pd.read_csv("DataSets/archive/annotations.csv")
annotation.tail()


Output/label/train
Output/label/valid
Output/label/test
Output/images2/train
Output/images2/valid
Output/images2/test


,image,xmin,ymin,xmax,ymax,label
2335,image-69.png,227.765502,207.710259,256.000000,248.054115,rbc
2336,image-69.png,201.832018,211.930101,251.736189,256.000000,rbc
2337,image-69.png,81.188275,231.883878,130.855693,254.391206,rbc
2338,image-69.png,35.903044,244.555806,79.863585,255.066516,rbc
2339,image-69.png,176.896280,244.360767,215.302142,255.741826,rbc


In [22]:
images = annotation.image.unique()
np.random.shuffle(images)
train_images = images[:64]
valid_images = images[64:80]
test_images = images[80:]

pre_path = 'DataSets/archive/images/'
prep = 'Output/'
for im in images:
    image = cv2.imread(pre_path + im)  
    cv2.imwrite(prep+im, image)  

In [10]:
train_images = train_images.tolist()


wbc_labels = []
wbc_images = []
new_images = []
for image_name in train_images:
    labels = annotation[(annotation['image'] == image_name) & (annotation['label'] == 'wbc')].values
    if len(labels) > 0 and np.random.rand() < 0.3:
        image = cv2.imread(pre_path + image_name)
        x1, y1, x2, y2 = labels[0, 1:5]
        x1, y1 = x1 * 1.05, y1 * 1.05
        x2, y2 = 0.95 * x2, 0.95 * y2
        if x2 < x1 or y2 < y1:
            continue

        c_image = image[int(y1):int(y2), int(x1):int(x2)]
        c_image = cv2.resize(c_image, (256, 256))
        wbc_images.append(c_image)
        new_labels = labels.copy()
        new_labels[0, [1, 2, 3, 4]] = 0.001, 001.001, 255.999, 255.999
        new_labels[0, 0] = 'w' + new_labels[0, 0]
        wbc_labels.append(new_labels[0])
        cv2.imwrite(prep+new_labels[0, 0], c_image)
        new_images.append(new_labels[0, 0])

train_images.extend(new_images)
print(len(train_images))
wbc_labels = np.stack(wbc_labels)
array_df = pd.DataFrame(wbc_labels, columns=annotation.columns)
annotation = pd.concat([annotation, array_df], axis=0, ignore_index=True)
len(annotation)

78


2354

In [11]:
all_labels = []
tmp = []

for image_name in train_images:
#     if image_name[0] != 'w':
    image = cv2.imread(prep + image_name)
    imagev = np.fliplr(image)
    imageh = np.flipud(image)
    imagevh = np.fliplr(np.flipud(image))
    labels = annotation[annotation['image'] == image_name].values

    b = 1
    labels_h = labels.copy()
    labels_v = labels.copy()
    labels_vh = labels.copy()

    labels_h[:, 0] = 'h'+labels_h[:, 0]
    labels_v[:, 0] = 'v' + labels_v[:, 0]
    labels_vh[:, 0] = 'vh' + labels_vh[:, 0]

    labels_h[:, [b+0, b+2]] = 256 - labels[:, [b+2, b+0]]
    labels_v[:, [b+1, b+3]] = 256 - labels[:, [b+3, b+1]]
    labels_vh[:, [b+0, b+1, b+2, b+3]] = 256 - labels[:, [b+2, b+3, b+0, b+1]]

    new_labels = np.vstack((labels_h, labels_v, labels_vh))
    all_labels.append(new_labels)
#     print(image_name)
    tmp.append(labels_h[:, 0][0])
    tmp.append(labels_v[:, 0][0])
    tmp.append(labels_vh[:, 0][0])

    cv2.imwrite(prep+labels_h[:, 0][0], imagev)
    cv2.imwrite(prep+labels_v[:, 0][0], imageh)
    cv2.imwrite(prep+labels_vh[:, 0][0], imagevh)

train_images.extend(tmp)
all_labels = np.vstack(all_labels)

array_df = pd.DataFrame(all_labels, columns=annotation.columns)
annotation = pd.concat([annotation, array_df], axis=0, ignore_index=True)
print(len(annotation))

6809


In [12]:
train_images = np.array(train_images)

In [14]:
import glob
from tqdm import tqdm
import os
import cv2
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [15]:
WIDTH = 256
HEIGHT = 256
cells_id = {
    'rbc': 0,
    'wbc': 1
}

In [16]:
cells_classes = list(cells_id.keys())
cells_classes

['rbc', 'wbc']

In [18]:
def save_labels(images_set, subfolder):
    for image in images_set:
        lables_file = os.path.join(label_dir, subfolder, image.replace(".png", ".txt"))
        with open(lables_file, "w") as wobj:
            for box in annotation.loc[annotation.image == image].values:
                wobj.write("%d %f %f %f %f \n" % (
                    cells_id[box[5]],
                    ((box[3]+box[1])/2.0) / WIDTH,
                    ((box[4]+box[2])/2.0) / HEIGHT,
                    (box[3]-box[1]) / WIDTH,
                    (box[4]-box[2]) / HEIGHT
                ))

                
save_labels(train_images, 'train')
save_labels(valid_images, 'valid')
save_labels(test_images, 'test')

In [23]:
SIZE = (256, 256)
def replace_image(src_file, dst_file, size=SIZE):
    image = cv2.imread(src_file) 
    image = cv2.resize(image, size)
    cv2.imwrite(dst_file, image)

def save_images(images_set, subfolder):
    for image in images_set:
        src_file = os.path.join(prep , image)
        dst_file = os.path.join(image_dir, subfolder, image)
        replace_image(src_file, dst_file)
        

        
save_images(train_images, 'train')
save_images(valid_images, 'valid')
save_images(test_images, 'test')

In [41]:
%%writefile data/cell_detection.yaml
train: Output/images2/train
val: Output/images2/valid
test: Output/images2/test
 
# Classes
nc: 2  # number of classes    
names: ['rbc', 'wbc']

Writing data/cell_detection.yaml


In [42]:
!curl -L -o yolov7_training.pt https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 72.1M  100 72.1M    0     0  12.1M      0  0:00:05  0:00:05 --:--:-- 16.2M


In [32]:
import wandb
wandb.login()

/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/vighneshms/.netrc


True

In [44]:
!python3 train.py --epochs 90 --workers 4 --device cpu --batch-size 16 --data data/cell_detection.yaml \
--img 256 --weights 'yolov7_training.pt' \
--name yolov7_BCD_fixed_res --hyp data/hyp.scratch.custom.yaml

/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
YOLOR 🚀 0283696 torch 2.4.1 CPU

Namespace(weights='yolov7_training.pt', cfg='', data='data/cell_detection.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=90, batch_size=16, img_size=[256, 256], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='cpu', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='yolov7_BCD_fixed_res', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='ru

In [46]:
!python3 test.py --weights runs/train/yolov7_BCD_fixed_res/weights/best.pt \
--task test --data data/cell_detection.yaml 

/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Namespace(weights=['runs/train/yolov7_BCD_fixed_res/weights/best.pt'], data='data/cell_detection.yaml', batch_size=32, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 0283696 torch 2.4.1 CPU

Traceback (most recent call last):
  File "/Users/vighneshms/Downloads/Red_cell.Ai/utils/google_utils.py", line 26, in attempt_download
    assets = [x['name'] for x in response['assets']]  # release assets
KeyError: 'assets'

During handling of the above exception, another exception occurred:

Trac

wandb-core(71585) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71586) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71599) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71602) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71603) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71604) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71605) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71607) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb-core(71608) MallocStackLogging: can't turn off malloc stack logging because 